In [1]:
import os
import json
import pandas as pd
import numpy as np
import requests
from modules.transcript_dataset import TranscriptDataset

In [2]:
# Please set a valid Google API
# Please download the 'data' folder from https://drive.google.com/drive/folders/1D-OHX0LUNA5Y-0h8UXwqa4H_yETZN_t_ to your project root directory

GOOGLE_API_KEY = '********************'

In [3]:
dataset_builder = TranscriptDataset(api_key=GOOGLE_API_KEY)
dataset_builder.build_dataset(dataset_folder='data', videos_count=2000)
dataset_builder.build_comments(dataset_folder='data', comments_count=200)

Working on file dataset_1.json
Fetching comments of MMbdcJhNxtI
Fetching comments of TgWyEsZX48s
Fetching comments of US4nNVZ7wO8
Fetching comments of Zi8Z6eEDLxM
Fetching comments of _V6TZulh1G0
Fetching comments of cieFkhp6aNw
Fetching comments of f5tbu9KX7j4
Fetching comments of kTrhlDVdIoQ
Fetching comments of tLSeYR6qOlM
Fetching comments of vE7G6uNk3Rw
Fetching comments of wC4VE-C8ziw
Working on file dataset_0.json
Fetching comments of -0ez6DTOjAw
Fetching comments of -3p1uftNTjg
Fetching comments of -dexXrCAPA0
Fetching comments of 0OkxaLKLeho
Fetching comments of 0ciWr7cpcHA
Fetching comments of 0kRR2k37zxw
Fetching comments of 0zDpTec602U
Fetching comments of 14esfF2kzWQ
Fetching comments of 1Cl86pTIedc
Fetching comments of 1CmITM6hu8w
Fetching comments of 1NZZtIDJtas
Fetching comments of 1Pql2nNR80k
Fetching comments of 1UvKkrihnMc
Fetching comments of 1_IcUcUftvg
Fetching comments of 1urVTL5DeRM
Fetching comments of 1zODrIa-eS0
Fetching comments of 21C5NaXroP0
Fetching comme